In [1]:
import psycopg2
import os
import pandas as pd
from io import BytesIO, StringIO

from sqlalchemy import create_engine
from sqlalchemy import types as sqltype
from config.database import HOST, PORT, USER, PASSWORD, DATABASE

import gc

In [2]:
os.chdir("/mnt/data0/proj_osgeo/data")
db_config=open('./db_config')
dbHOST=db_config.readline().split('\"')[1]
dbUSER=db_config.readline().split('\"')[1]
dbPASS=db_config.readline().split('\"')[1]
dbDB=db_config.readline().split('\"')[1]
db_config.close()
conn = psycopg2.connect(host=dbHOST,user=dbUSER,password=dbPASS,database=dbDB)
conn.commit()
cur = conn.cursor()

In [5]:
def escape(s):
    return (s).replace("'", "''").replace('%', '%%').replace('<','').replace('>','')

In [3]:
psql_engine = create_engine("postgresql://"+USER+":"+PASSWORD+"@"+HOST+":"+str(PORT)+"/"+DATABASE)

In [12]:
DIR = './raw'
DIR_csv = './csv'
filelist_in=os.listdir(DIR)

#projs = set()
#cur.execute("select proj_id from project")
#result=cur.fetchall()
#for proj_id in result:
#    proj_id = proj_id[0].replace(' ','')
#    print(proj_id)
#    projs.add(proj_id)


for element in filelist_in:
    csv_path = DIR_csv+"/"+element+'/'
    csv_file = csv_path+element+"_all.csv"
#data1 = pd.DataFrame(data)
# dataframe类型转换为IO缓冲区中的str类型

    data1 = pd.read_csv(csv_file, index_col= 0)
    data1 = data1.loc[:,['message_id', 'thread_id', 'author_aliase_id', 'author_name', 'receivers_name', 'timestamp', 'message_text' ]]
    data1['timestamp'] = data1['timestamp'].apply(lambda x: pd.Timestamp(x))
    #data1.fillna("None")
    print("Project: ")
    df_psql = data1.astype(str)
    for col in df_psql.columns.values :
        df_psql[col]= df_psql[col].apply(lambda x : escape(x).encode('utf-8','ignore').decode("utf-8"))
    df_psql.to_sql(name='message', con = psql_engine, if_exists= 'append', index= False,chunksize=1)


Project: 


IntegrityError: (psycopg2.errors.ForeignKeyViolation) insert or update on table "message" violates foreign key constraint "message_thread_id_fkey"
DETAIL:  Key (thread_id)=(nan) is not present in table "thread".

[SQL: INSERT INTO message (message_id, thread_id, author_aliase_id, author_name, receivers_name, timestamp, message_text) VALUES (%(message_id)s, %(thread_id)s, %(author_aliase_id)s, %(author_name)s, %(receivers_name)s, %(timestamp)s, %(message_text)s)]
[parameters: {'message_id': '493401EB.4080304@comai.to', 'thread_id': 'nan', 'author_aliase_id': 'Stefano Bonnin_stefano.bonnin@comai.to ', 'author_name': 'Stefano Bonnin', 'receivers_name': 'nan', 'timestamp': '2010-09-01 17:24:05', 'message_text': "From: stefano.bonnin at comai.to (Stefano Bonnin)\nDate: Wed Sep  1 17:24:05 2010\nSubject: [OpenLayers-Dev] Ticket #1666\nMessage-ID: 493401EB.40803 ... (83 characters truncated) ... t \n#1666 progress?\nThanks in advance.\n\n-- \nStefano Bonnin\nComai Torino S.r.l\nVia Livorno 60

In [4]:
DIR = './raw'
DIR_csv = './csv'
filelist_in=os.listdir(DIR)

projs = set()
cur.execute("select proj_id from project")
result=cur.fetchall()
for proj_id in result:
    print(proj_id)
    projs.add(proj_id[0])


for element in filelist_in:
    csv_path = DIR_csv+"/"+element+'/'
    csv_file = csv_path+element+"_all.csv"

    

Index(['message_id', 'thread_id', 'author_aliase_id', 'author_name',
       'receivers_name', 'timestamp', 'message_text'],
      dtype='object')

In [ ]:
data1["message_text"][2]